In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib widget

from toolz import *

import os
import pickle
import json
import pathlib

In [26]:
lmap = compose(list, map)

In [69]:
def list_of_str_to_list_of_json(lst):
    it = map(lambda x: x.replace('\xa0', ' '), lst)
    it = filter(lambda x: x != '', it)
    it = lmap(json.loads, it)
    return it

In [84]:
data_path = pathlib.Path('data')

In [104]:
res = {}
for elt in data_path.iterdir():
    res[str(elt)] = pd.json_normalize(json.loads(elt.read_text(encoding='utf-8')))

In [115]:
res_lst = list(res.values())
res_lst[0]

,title,menu,description,category,cuisine
0,Маффины с шоколадом,Еда без ограничений,,Выпечка и десерты,Итальянская кухня
1,Быстрый тирамису с малиной и мятой,Еда без ограничений,Один из самых популярных итальянских десертов ...,Выпечка и десерты,Итальянская кухня
2,Тирамису с сыром маскарпоне,Еда без ограничений,За трудоемкую и не слишком быструю классическу...,Выпечка и десерты,Итальянская кухня
3,Торт «Прага» по старинному рецепту,Еда без ограничений,,Выпечка и десерты,Итальянская кухня
4,Морковный торт с грецкими орехами и корицей,Еда без ограничений,,Выпечка и десерты,Итальянская кухня
...,...,...,...,...,...
11740,Вафли на йогурте,Еда без ограничений,,Выпечка и десерты,Бельгийская кухня
11741,Сырные шарики с виноградом,Еда без ограничений,,Выпечка и десерты,Бельгийская кухня
11742,Печенье альфахорес (alfajores),Еда без ограничений,,Выпечка и десерты,Аргентинская кухня
11743,Аргентинский шоколадный торт,Еда без ограничений,,Выпечка и десерты,Аргентинская кухня


In [121]:
df_concatenated = pd.concat(res_lst).reset_index(drop=True)
df_concatenated

,title,menu,description,category,cuisine
0,Маффины с шоколадом,Еда без ограничений,,Выпечка и десерты,Итальянская кухня
1,Быстрый тирамису с малиной и мятой,Еда без ограничений,Один из самых популярных итальянских десертов ...,Выпечка и десерты,Итальянская кухня
2,Тирамису с сыром маскарпоне,Еда без ограничений,За трудоемкую и не слишком быструю классическу...,Выпечка и десерты,Итальянская кухня
3,Торт «Прага» по старинному рецепту,Еда без ограничений,,Выпечка и десерты,Итальянская кухня
4,Морковный торт с грецкими орехами и корицей,Еда без ограничений,,Выпечка и десерты,Итальянская кухня
...,...,...,...,...,...
40937,Пастель де карне — аргентинский мясной рулет,Еда без ограничений,,Основные блюда,Аргентинская кухня
40938,Локро — рагу по-аргентински,Еда без ограничений,,Основные блюда,Аргентинская кухня
40939,Фиеста таурино (Fiesta Taurino),Еда без ограничений,,Основные блюда,Аргентинская кухня
40940,Аргентинское рагу с фасолью и копченостями,Еда без ограничений,,Основные блюда,Аргентинская кухня


In [124]:
df_with_description = df_concatenated[df_concatenated['description'] != '']#.reset_index(drop=True)
df_with_description

,title,menu,description,category,cuisine
1,Быстрый тирамису с малиной и мятой,Еда без ограничений,Один из самых популярных итальянских десертов ...,Выпечка и десерты,Итальянская кухня
2,Тирамису с сыром маскарпоне,Еда без ограничений,За трудоемкую и не слишком быструю классическу...,Выпечка и десерты,Итальянская кухня
7,Шоколадные трюфели в какао,Еда без ограничений,Классический шоколадный трюфель с нежной кремо...,Выпечка и десерты,Итальянская кухня
15,Фисташковый мусс с авокадо,Вегетарианская еда,Этот эффектный мусс невероятно прост в пригото...,Выпечка и десерты,Итальянская кухня
16,Тирамису,Еда без ограничений,"Кажется, что этому десерту — сотни лет, но на ...",Выпечка и десерты,Итальянская кухня
...,...,...,...,...,...
40900,Стейк рибай с черным перцем,Еда без ограничений,Рецепт взят из книги Ники Ганич «География на ...,Основные блюда,Эстонская кухня
40912,Кролик с черносливом,Еда без ограничений,Теплое ламповое блюдо: тушеный кролик с чернос...,Основные блюда,Бельгийская кухня
40914,Мясной хлеб,Еда без ограничений,Мясной хлеб — древнее блюдо: впервые он упомин...,Основные блюда,Бельгийская кухня
40923,Карбонад по-фламандски,Еда без ограничений,Carbonnades flamandes,Основные блюда,Бельгийская кухня


In [127]:
df_with_description.groupby('category').count() / df_concatenated.groupby('category').count()

,title,menu,description,cuisine
category,,,,
Выпечка и десерты,0.075266,0.075266,0.075266,0.075266
Завтраки,0.125061,0.125061,0.125061,0.125061
Закуски,0.115444,0.115444,0.115444,0.115444
Напитки,0.174377,0.174377,0.174377,0.174377
Основные блюда,0.094600,0.094600,0.094600,0.094600
Паста и пицца,0.057143,0.057143,0.057143,0.057143
Салаты,0.080799,0.080799,0.080799,0.080799
Соусы и маринады,0.082005,0.082005,0.082005,0.082005
Супы,0.133968,0.133968,0.133968,0.133968
